In [4]:
import os, sys
print("CWD:", os.getcwd())
print("Here:", os.listdir())


CWD: C:\cv_partb
Here: ['.git', '.gitattributes', '.gitignore', '.ipynb_checkpoints', 'data', 'data_raw', 'diagnostics.ipynb', 'labels.json', 'notebooks', 'outputs', 'README.md', 'requirements.txt', 'streamlit_app.py']


In [5]:
import json, os
with open("labels.json", "r", encoding="utf-8") as f:
    labels = json.load(f)
print("labels.json (len={}):".format(len(labels)), labels)


labels.json (len=4): ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy']


In [6]:
import tensorflow as tf
MODEL_PATH = os.path.join("outputs","checkpoints","modelB_phase2_best.keras")
model = tf.keras.models.load_model(MODEL_PATH)
model.summary()  # look at the LAST Dense: units must be 4
print("Model output shape:", model.output_shape)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide (TrueDivide)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract (Subtract)                  │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │           5,124 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,636,430 (21.50 MB)

 Trainable params: 1,686,660 (6.43 MB)

 Non-trainable params: 576,448 (2.20 MB)

 Optimizer params: 3,373,322 (12.87 MB)

Model output shape: (None, 4)


In [7]:
# Paths (edit if yours are different)
MODEL_PATH = r"outputs/checkpoints/modelB_phase2_best.keras"
LABELS_PATH = r"labels.json"

import os, sys, json, glob, pathlib, textwrap, hashlib
from pathlib import Path
import numpy as np
from PIL import Image

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

print("TF version:", tf.__version__)
print("Working dir:", os.getcwd())
print("MODEL_PATH exists?", os.path.exists(MODEL_PATH))
print("LABELS_PATH exists?", os.path.exists(LABELS_PATH))


TF version: 2.17.0
Working dir: C:\cv_partb
MODEL_PATH exists? True
LABELS_PATH exists? True


In [8]:
with open(LABELS_PATH, "r", encoding="utf-8") as f:
    labels = json.load(f)

print("labels.json len =", len(labels))
print("labels.json =", labels)

# Helpful: show one-class-per-line so you can visually inspect the order.
for i, name in enumerate(labels):
    print(f"{i}: {name}")


labels.json len = 4
labels.json = ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy']
0: Tomato_Bacterial_spot
1: Tomato_Early_blight
2: Tomato_Late_blight
3: Tomato_healthy


In [9]:
m = tf.keras.models.load_model(MODEL_PATH)
m.summary()
print("Model output shape:", m.output_shape)
assert m.output_shape[-1] == len(labels), \
    f"Mismatch! Model outputs {m.output_shape[-1]} but labels.json has {len(labels)}"
print("OK: Model output units == len(labels)")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide (TrueDivide)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract (Subtract)                  │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │           5,124 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,636,430 (21.50 MB)

 Trainable params: 1,686,660 (6.43 MB)

 Non-trainable params: 576,448 (2.20 MB)

 Optimizer params: 3,373,322 (12.87 MB)

Model output shape: (None, 4)
OK: Model output units == len(labels)


In [10]:
DATA_DIR_TRAIN = Path("data/train")  # <-- EDIT if different

if DATA_DIR_TRAIN.exists():
    classes_found = sorted([p.name for p in DATA_DIR_TRAIN.iterdir() if p.is_dir()])
    print("Folders under", DATA_DIR_TRAIN, "=>", classes_found, "len=", len(classes_found))
else:
    print("WARN: DATA_DIR_TRAIN not found. Edit DATA_DIR_TRAIN to the correct path.")


Folders under data\train => ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy'] len= 4


In [11]:
from random import choice

def preprocess_pil(img_pil, size=(224,224)):
    img = img_pil.convert("RGB").resize(size)
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = preprocess_input(x)  # MobileNetV2 preprocess
    return np.expand_dims(x, 0)

def predict_path(model, labels, path):
    img = Image.open(path)
    x = preprocess_pil(img, size=(224,224))
    probs = model.predict(x, verbose=0)[0]
    idx = int(np.argmax(probs))
    top = labels[idx]
    conf = float(probs[idx])
    dist = {labels[i]: float(probs[i]) for i in range(len(labels))}
    return top, conf, dist

# Try to locate test images under data/test/<class>/... or data_raw/<class>/...
CANDIDATE_TEST_DIRS = [Path("data/test"), Path("data_raw"), Path("data/val"), Path("data")]
found_any = False

for base in CANDIDATE_TEST_DIRS:
    if not base.exists():
        continue
    print("\nScanning:", base)
    for cls in labels:
        class_dir = base / cls
        if class_dir.exists():
            files = [str(p) for p in class_dir.glob("*") if p.suffix.lower() in {".jpg",".jpeg",".png",".bmp"}]
            if files:
                found_any = True
                path = choice(files)
                top, conf, dist = predict_path(m, labels, path)
                print(f"\nClass: {cls}\nFile: {path}\nPred → {top}  (conf={conf:.3f})")
                print(dist)

if not found_any:
    print("No test images found under known locations. Point CANDIDATE_TEST_DIRS to your test folders.")



Scanning: data\test

Class: Tomato_Bacterial_spot
File: data\test\Tomato_Bacterial_spot\887eab5c-9475-46b7-81df-ea2c5190e7fe___GCREC_Bact.Sp 6026.JPG
Pred → Tomato_Early_blight  (conf=0.974)
{'Tomato_Bacterial_spot': 1.4047498098079814e-07, 'Tomato_Early_blight': 0.974284827709198, 'Tomato_Late_blight': 0.025506658479571342, 'Tomato_healthy': 0.00020833595772273839}

Class: Tomato_Early_blight
File: data\test\Tomato_Early_blight\484250ea-cc2c-4614-bee8-7f24007e00ac___RS_Erly.B 9463.JPG
Pred → Tomato_Early_blight  (conf=0.978)
{'Tomato_Bacterial_spot': 1.3557333033986652e-07, 'Tomato_Early_blight': 0.9779013395309448, 'Tomato_Late_blight': 0.021934427320957184, 'Tomato_healthy': 0.00016405197675339878}

Class: Tomato_Late_blight
File: data\test\Tomato_Late_blight\df7775b2-73f3-4708-86d5-a56e2bbb3337___RS_Late.B 5378.JPG
Pred → Tomato_Early_blight  (conf=0.974)
{'Tomato_Bacterial_spot': 2.399609968506411e-07, 'Tomato_Early_blight': 0.9741044640541077, 'Tomato_Late_blight': 0.02566249296

In [9]:
APP_PATH = Path("streamlit_app.py")
if APP_PATH.exists():
    app_code = APP_PATH.read_text(encoding="utf-8", errors="ignore")
    # Show the model path & preprocess calls found
    for key in ["MODEL_PATH", "preprocess_input", "resize", "labels.json", "checkpoints", "predict("]:
        if key in app_code:
            print(f"[FOUND] {key}")
    print("\n--- snippet (first 200 lines) ---\n")
    print("\n".join(app_code.splitlines()[:200]))
else:
    print("streamlit_app.py not found here")


[FOUND] MODEL_PATH
[FOUND] preprocess_input
[FOUND] resize
[FOUND] labels.json
[FOUND] checkpoints
[FOUND] predict(

--- snippet (first 200 lines) ---

import os, json
import numpy as np
import streamlit as st
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image as kimage

MODEL_PATH  = os.path.join("outputs", "checkpoints", "modelB_phase2_best.keras")
LABELS_PATH = "labels.json"
IMG_SIZE    = (224, 224)
MIN_BYTES   = 1_000_000
# ---------------------------------------

def ensure_dirs():
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

def ensure_model_local():
    """
    Ensure the model file exists locally.
    If missing or too small, prompt user to upload the .keras file (one-time).
    """
    ensure_dirs()
    needs_model = (not os.path.exists(MODEL_PATH)) or (os.path.getsize(MODEL_PATH) < MIN_BYTES)

    if needs_model:
        st.warning(
            "Model file is missing on this server. "
            "Please upload your 

In [10]:
def md5(path, blocksize=1<<20):
    m = hashlib.md5()
    with open(path, "rb") as f:
        while True:
            b = f.read(blocksize)
            if not b: break
            m.update(b)
    return m.hexdigest()

# Find all *.keras and all labels.json in repo (recursively)
keras_files = [str(p) for p in Path(".").rglob("*.keras")]
label_files = [str(p) for p in Path(".").rglob("labels.json")]

print("KERAS FILES:")
for p in keras_files:
    try:
        print(" ", p, "md5:", md5(p), "size:", os.path.getsize(p))
    except Exception as e:
        print(" ", p, " (error reading md5)", e)

print("\nLABELS.JSON FILES:")
for p in label_files:
    try:
        print(" ", p, "md5:", md5(p), "size:", os.path.getsize(p))
    except Exception as e:
        print(" ", p, " (error reading md5)", e)


KERAS FILES:
  outputs\checkpoints\modelA_best.keras md5: 704e69bdb657db0f81f55083ecd32330 size: 1171061
  outputs\checkpoints\modelA_best_v2.keras md5: 2d2e5ae8be1392978f1765cbc1e0cfd1 size: 1171289
  outputs\checkpoints\modelB_phase1_best.keras md5: 5285be26b355bad8b30280a1fb58afeb size: 9695221
  outputs\checkpoints\modelB_phase2_best.keras md5: db2f1876540b0c071387d2cb733c5402 size: 23177500

LABELS.JSON FILES:
  labels.json md5: a5264e7cb2be2cd92c28adec4eda7d26 size: 90


In [11]:
import random, collections

def sample_paths(base, cls, k=10):
    p = Path(base)/cls
    if not p.exists(): return []
    files = [str(x) for x in p.glob("*") if x.suffix.lower() in {".jpg",".jpeg",".png",".bmp"}]
    random.shuffle(files)
    return files[:k]

# Choose a base test dir that actually has images:
BASE = None
for c in CANDIDATE_TEST_DIRS:
    if (c/labels[0]).exists():
        BASE = c
        break

if BASE is None:
    print("No suitable test dir found; skip this cell or set BASE manually.")
else:
    print("Using test base:", BASE)
    counts = collections.Counter()
    total = 0
    for cls in labels:
        for path in sample_paths(BASE, cls, k=10):
            pred, conf, dist = predict_path(m, labels, path)
            counts[(cls, pred)] += 1
            total += 1

    # Pretty print
    print("\nMini confusion (counts):")
    header = "true\\pred".ljust(24) + "  " + "  ".join([p[:20].ljust(20) for p in labels])
    print(header)
    for t in labels:
        row = [str(counts.get((t,p),0)).ljust(20) for p in labels]
        print(t[:24].ljust(24), "  ", "  ".join(row))
    print("\nTotal samples:", total)


Using test base: data\test

Mini confusion (counts):
true\pred                 Tomato_Bacterial_spo  Tomato_Early_blight   Tomato_Late_blight    Tomato_healthy      
Tomato_Bacterial_spot       0                     10                    0                     0                   
Tomato_Early_blight         0                     10                    0                     0                   
Tomato_Late_blight          0                     10                    0                     0                   
Tomato_healthy              0                     10                    0                     0                   

Total samples: 40


In [12]:
# Load a test image from known class
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

img_path = "data/test/Tomato_Late_blight/2d583f2c-3e59-4c4c-be45-5f367a38af95___GHLB2 Leaf 8724.JPG"  # <-- CHANGE to any image
img = Image.open(img_path).convert("RGB").resize((224,224))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)

# Load model
model = tf.keras.models.load_model("outputs/checkpoints/modelB_phase2_best.keras")
pred = model.predict(x)[0]
print("Raw output:", pred)
print("Argmax:", np.argmax(pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Raw output: [2.2489110e-07 9.6904469e-01 3.0674538e-02 2.8054995e-04]
Argmax: 1


In [13]:
import json

with open("labels.json", "r") as f:
    labels = json.load(f)

print("labels.json =", labels)
for i, lbl in enumerate(labels):
    print(f"{i}: {lbl}")


labels.json = ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy']
0: Tomato_Bacterial_spot
1: Tomato_Early_blight
2: Tomato_Late_blight
3: Tomato_healthy


In [14]:
top_idx = np.argmax(pred)
print("Predicted label:", labels[top_idx])


Predicted label: Tomato_Early_blight


In [21]:
import os
base = "data/train"  # or wherever your training data folders are
labels = sorted(os.listdir(base))  # alphabetical order used by Keras generators
print(labels)
with open("labels.json", "w") as f:
    json.dump(labels, f)
print("✅ Saved labels.json")


['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy']
✅ Saved labels.json


In [22]:
import json
with open("labels.json","r",encoding="utf-8") as f:
    labels = json.load(f)
print("labels.json:", labels, "len =", len(labels))
for i,n in enumerate(labels): print(i, n)

labels.json: ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy'] len = 4
0 Tomato_Bacterial_spot
1 Tomato_Early_blight
2 Tomato_Late_blight
3 Tomato_healthy


In [21]:
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import json

# Load model and labels
model = tf.keras.models.load_model("outputs/checkpoints/modelB_phase2_best.keras")
with open("labels.json", "r") as f:
    labels = json.load(f)

# === STEP 1: CHANGE THE IMAGE PATH BELOW ===
img_path = "data/test/Tomato_Late_blight/1a473cad-42fc-48ca-963c-d438fbca928f___RS_Late.B 5418.JPG"

# === STEP 2: Prediction ===
img = Image.open(img_path).convert("RGB").resize((224, 224))
x = tf.keras.preprocessing.image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)

# Predict
pred = model.predict(x, verbose=0)[0]
predicted_index = int(np.argmax(pred))
predicted_label = labels[predicted_index]
confidence = float(pred[predicted_index]) * 100.0

# Output
print("✅ Prediction Result")
print("Image:", img_path)
print("Predicted Class:", predicted_label)
print("Confidence: {:.2f}%".format(confidence))
print("\nAll Class Probabilities:")
for i, prob in enumerate(pred):
    print(f"- {labels[i]}: {prob*100:.2f}%")


✅ Prediction Result
Image: data/test/Tomato_Late_blight/1a473cad-42fc-48ca-963c-d438fbca928f___RS_Late.B 5418.JPG
Predicted Class: Tomato_Late_blight
Confidence: 97.53%

All Class Probabilities:
- Tomato_Bacterial_spot: 0.00%
- Tomato_Early_blight: 2.47%
- Tomato_Late_blight: 97.53%
- Tomato_healthy: 0.00%


In [17]:
import tensorflow as tf, os, json
MODEL_PATH = os.path.join("outputs","checkpoints","modelB_phase2_best.keras")

model = tf.keras.models.load_model(MODEL_PATH)
print("Model output shape:", model.output_shape)   # expect (..., 4)

with open("labels.json","r",encoding="utf-8") as f:
    labels = json.load(f)
assert model.output_shape[-1] == len(labels), "Mismatch: model units vs label count!"
print("✅ Units match labels:", len(labels))


Model output shape: (None, 4)
✅ Units match labels: 4


In [18]:
import numpy as np, glob, random, os
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

IMG_SIZE = (224,224)

def preprocess_pil(img):
    img = img.convert("RGB").resize(IMG_SIZE)
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = preprocess_input(x)
    return np.expand_dims(x, 0)

def predict_path(path):
    img = Image.open(path)
    x = preprocess_pil(img)
    probs = model.predict(x, verbose=0)[0]
    idx = int(np.argmax(probs))
    return labels[idx], float(probs[idx]), dict(zip(labels, [float(p) for p in probs]))

# test images from data/test/<class>/... or data_raw/<class>/...
for cls in labels:
    candidates = glob.glob(os.path.join("data","test",cls,"*")) or glob.glob(os.path.join("data_raw",cls,"*"))
    if not candidates:
        print(f"[WARN] no test images for {cls}")
        continue
    p = random.choice(candidates)
    pred, conf, dist = predict_path(p)
    print(f"\nTrue: {cls}\nFile: {p}\nPred: {pred}  (conf={conf:.3f})")
    print(dist)



True: Tomato_Bacterial_spot
File: data\test\Tomato_Bacterial_spot\bf962eca-a579-4f9d-aab4-7406179290b9___GCREC_Bact.Sp 6283.JPG
Pred: Tomato_Early_blight  (conf=0.974)
{'Tomato_Bacterial_spot': 1.6923151235914702e-07, 'Tomato_Early_blight': 0.9736642241477966, 'Tomato_Late_blight': 0.026117566972970963, 'Tomato_healthy': 0.00021805161668453366}

True: Tomato_Early_blight
File: data\test\Tomato_Early_blight\484250ea-cc2c-4614-bee8-7f24007e00ac___RS_Erly.B 9463.JPG
Pred: Tomato_Early_blight  (conf=0.978)
{'Tomato_Bacterial_spot': 1.3557333033986652e-07, 'Tomato_Early_blight': 0.9779013395309448, 'Tomato_Late_blight': 0.021934427320957184, 'Tomato_healthy': 0.00016405197675339878}

True: Tomato_Late_blight
File: data\test\Tomato_Late_blight\4b115bc3-911c-4e98-b948-a6a360f06a87___GHLB_PS Leaf 25 Day 9.jpg
Pred: Tomato_Early_blight  (conf=0.968)
{'Tomato_Bacterial_spot': 2.5620761334721465e-07, 'Tomato_Early_blight': 0.9675661325454712, 'Tomato_Late_blight': 0.0322280116379261, 'Tomato_hea

In [19]:
# Inspect one batch from the training dataset
for bx, by in train_ds.take(1):
    print("X shape:", bx.shape, bx.dtype)
    print("Y shape:", by.shape, by.dtype)
    # Peek at first 3 labels
    print("First 3 labels:\n", by[:3].numpy())
    break


NameError: name 'train_ds' is not defined

In [20]:
# ==== ONE-CELL FIX: retrain correctly on 4 classes, save model+labels, sanity-check ====
import os, json, pathlib, numpy as np, tensorflow as tf
from pathlib import Path
from PIL import Image
from tensorflow.keras import layers, Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# -----------------------------
# 0) CONFIG – EDIT THESE PATHS
# -----------------------------
DATA_TRAIN = Path("data/train")  # e.g. data/train/<class>/*
DATA_VAL   = Path("data/val")    # e.g. data/val/<class>/*
DATA_TEST  = Path("data/test")   # e.g. data/test/<class>/*
TEST_IMG   = Path("data/test/Tomato_Late_blight/1a473cad-42fc-48ca-963c-d438fbca928f___RS_Late.B 5418.JPG")  # change to any image you want to test

# Fixed class list (keeps order consistent everywhere)
CLASS_NAMES = [
    "Tomato_Bacterial_spot",
    "Tomato_Early_blight",
    "Tomato_Late_blight",
    "Tomato_healthy",
]

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_PHASE1 = 8
EPOCHS_PHASE2 = 10
FINE_TUNE_UNFREEZE = 40  # unfreeze last N layers of MobileNetV2 in Phase 2
CKPT_DIR = Path("outputs/checkpoints"); CKPT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_OUT = CKPT_DIR / "modelB_phase2_best.keras"
LABELS_PATH = Path("labels.json")

# ---------------------------------------
# 1) SAVE labels.json (truth source)
# ---------------------------------------
with open(LABELS_PATH, "w", encoding="utf-8") as f:
    json.dump(CLASS_NAMES, f, indent=2)
print("✅ Saved labels.json:", CLASS_NAMES)

# ------------------------------------------------------
# 2) Build tf.data datasets with ONE-HOT labels
#    (label_mode='categorical' => matches softmax + categorical_crossentropy)
# ------------------------------------------------------
def make_ds(root: Path, shuffle=True):
    if not root.exists():
        return None
    ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory=str(root),
        labels="inferred",
        label_mode="categorical",              # ONE-HOT labels
        class_names=CLASS_NAMES,               # FORCE this exact order
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        interpolation="bilinear",
    )
    # Map MobileNetV2 preprocess ([-1,1]) to keep train==serve
    ds = ds.map(lambda x, y: (preprocess_input(tf.cast(x, tf.float32)), y),
                num_parallel_calls=tf.data.AUTOTUNE)
    return ds.prefetch(tf.data.AUTOTUNE)

train_ds = make_ds(DATA_TRAIN, shuffle=True)
val_ds   = make_ds(DATA_VAL,   shuffle=False)
test_ds  = make_ds(DATA_TEST,  shuffle=False)

if train_ds is None or val_ds is None:
    raise SystemExit("❌ DATA_TRAIN or DATA_VAL not found. Please set DATA_TRAIN/DATA_VAL paths correctly.")

print("✅ Datasets ready.")
print("Train classes (forced order):", CLASS_NAMES)

# -------------------------------------------------------------------
# 3) Build MobileNetV2 model (ImageNet) + 4-class softmax head
# -------------------------------------------------------------------
base = MobileNetV2(input_shape=(224,224,3), include_top=False, weights="imagenet")
base.trainable = False  # Phase 1: feature extraction

inputs = layers.Input(shape=(224,224,3))
# NOTE: We already preprocessed in tf.data; so feed inputs directly.
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)  # you used 0.3
outputs = layers.Dense(len(CLASS_NAMES), activation="softmax")(x)
model = Model(inputs, outputs)

# Compile with CORRECT loss for one-hot labels
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

ckpt_phase1 = CKPT_DIR / "modelB_phase1_best.keras"
cb1 = [
    tf.keras.callbacks.ModelCheckpoint(str(ckpt_phase1), monitor="val_accuracy",
                                       mode="max", save_best_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max",
                                     patience=3, restore_best_weights=True),
]

print("🔧 Phase 1 training (feature extraction)…")
hist1 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_PHASE1, callbacks=cb1, verbose=1)

# -----------------------------------------------------------
# 4) Fine-tune: unfreeze last N layers of the backbone
# -----------------------------------------------------------
base.trainable = True
# Freeze all but last N layers
for layer in base.layers[:-FINE_TUNE_UNFREEZE]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),  # smaller LR
              loss="categorical_crossentropy",
              metrics=["accuracy"])

cb2 = [
    tf.keras.callbacks.ModelCheckpoint(str(MODEL_OUT), monitor="val_accuracy",
                                       mode="max", save_best_only=True, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max",
                                     patience=4, restore_best_weights=True),
]

print("🔧 Phase 2 training (fine-tuning)…")
hist2 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_PHASE2, callbacks=cb2, verbose=1)

# --------------------------------------------
# 5) Save final model (already saved best)
# --------------------------------------------
print(f"✅ Saved best model to: {MODEL_OUT}")

# --------------------------------------------
# 6) Sanity: check model units == labels
# --------------------------------------------
fixed = tf.keras.models.load_model(str(MODEL_OUT))
assert fixed.output_shape[-1] == len(CLASS_NAMES), \
    f"Output units {fixed.output_shape[-1]} != {len(CLASS_NAMES)} labels"
print("✅ Model head units match labels:", fixed.output_shape[-1])

# --------------------------------------------
# 7) Single-image sanity prediction (edit TEST_IMG)
# --------------------------------------------
if TEST_IMG.exists():
    img = Image.open(TEST_IMG).convert("RGB").resize(IMG_SIZE)
    arr = tf.keras.preprocessing.image.img_to_array(img)
    # NOTE: train pipeline already used preprocess_input, so we must do the same here:
    arr = preprocess_input(arr)
    arr = np.expand_dims(arr, axis=0)
    probs = fixed.predict(arr, verbose=0)[0]
    idx = int(np.argmax(probs))
    print("\n✅ Single-Image Prediction")
    print("Image:", str(TEST_IMG))
    print("Predicted:", CLASS_NAMES[idx], "| Confidence: {:.2f}%".format(float(probs[idx])*100))
    print("All probs:", {CLASS_NAMES[i]: float(probs[i]) for i in range(len(CLASS_NAMES))})
else:
    print(f"⚠️ TEST_IMG not found: {TEST_IMG}")

# --------------------------------------------
# 8) Optional: quick test-set accuracy (if DATA_TEST exists)
# --------------------------------------------
if test_ds is not None:
    loss, acc = fixed.evaluate(test_ds, verbose=0)
    print(f"\n📊 Quick test set: acc={acc*100:.2f}%  loss={loss:.4f}")
else:
    print("ℹ️ No test dataset found; skipped quick evaluation.")


✅ Saved labels.json: ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy']
Found 4636 files belonging to 4 classes.
Found 993 files belonging to 4 classes.
Found 997 files belonging to 4 classes.
✅ Datasets ready.
Train classes (forced order): ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy']
🔧 Phase 1 training (feature extraction)…
Epoch 1/8
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 691ms/step - accuracy: 0.6291 - loss: 0.8890
Epoch 1: val_accuracy improved from None to 0.89527, saving model to outputs\checkpoints\modelB_phase1_best.keras
145/145 ━━━━━━━━━━━━━━━━━━━━ 133s 875ms/step - accuracy: 0.7640 - loss: 0.6039 - val_accuracy: 0.8953 - val_loss: 0.3014
Epoch 2/8
145/145 ━━━━━━━━━━━━━━━━━━━━ 0s 700ms/step - accuracy: 0.8922 - loss: 0.3190
Epoch 2: val_accuracy improved from 0.89527 to 0.92850, saving model to outputs\checkpoints\modelB_phase1_best.keras
145/145 ━━━━━━━━━━━━━━━━━━━━ 124s 853ms/step - accuracy: 0.9014 -